In [31]:
import pandas as pd 
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults,GetUpdatedPropertyDetails
from googlesearch import search
import pprint
import locale
import requests
from bs4 import BeautifulSoup
import requests
import json
import os
from dotenv import load_dotenv
import pprint
import xlrd
import numpy as np

In [32]:
load_dotenv()

True

In [15]:
#df = pd.read_csv('../data/input/api_keys.csv')
#zwsid = df.loc[df['API'] == 'zillow']['KEY'].iloc[0]
X_RapidAPI_Key=os.getenv('X_RapidAPI_Key')
propertyurl = "https://www.zillow.com/homedetails/4713-3rd-Pl-NW-Washington-DC-20011/486979_zpid/"
propertyurl

'https://www.zillow.com/homedetails/4713-3rd-Pl-NW-Washington-DC-20011/486979_zpid/'

In [10]:
zpid=[x for x in propertyurl.split('/')if 'zpid' in x][0].split('_')[0]
type(zpid)
print ('Zpid of the property is:',{zpid})

Zpid of the property is: {'486979'}


In [11]:
url = "https://zillow56.p.rapidapi.com/property"

querystring = {"zpid":zpid}

headers = {
	"X-RapidAPI-Key": X_RapidAPI_Key,
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)
response.json()

{'abbreviatedAddress': '4713 3rd Pl NW',
 'address': {'city': 'Washington',
  'community': None,
  'neighborhood': None,
  'state': 'DC',
  'streetAddress': '4713 3rd Pl NW',
  'subdivision': None,
  'zipcode': '20011'},
 'apartmentsForRentInZipcodeSearchUrl': {'path': '/washington-dc-20011/apartments/'},
 'attributionInfo': {'agentEmail': 'dcmultifamily@greysteel.com',
  'agentLicenseNumber': None,
  'agentName': 'Ari Firoozabadi',
  'agentPhoneNumber': '202-280-2722',
  'attributionTitle': None,
  'brokerName': 'Greysteel Company LLC',
  'brokerPhoneNumber': '202-280-2722',
  'buyerAgentMemberStateLicense': None,
  'buyerAgentName': None,
  'buyerBrokerageName': None,
  'coAgentLicenseNumber': None,
  'coAgentName': None,
  'coAgentNumber': None,
  'infoList1': [{'infoField1': 'listOffice',
    'infoField2': 'Greysteel Company LLC'}],
  'infoList2': [{'infoField1': 'Ari Firoozabadi',
    'infoField2': 'listAgent',
    'infoField3': 'PB98361353'}],
  'infoString1': 'DCDC2079820',
  'i

In [12]:
df_property_detail = pd.json_normalize(data=response.json())
print('Num of rows:', len(df_property_detail))
print ('Num of columns:',len(df_property_detail.columns))
df_property_detail.head()

Num of rows: 1
Num of columns: 551


,abbreviatedAddress,bathrooms,bedrooms,big,boroughId,boroughSearchUrl,brokerId,brokerIdDimension,brokerageName,building,...,topNavJson.topnav.json.main.sections,topNavJson.topnav.json.marketing.sections,topNavJson.topnav.json.regLogin.sections,tourEligibility.isPropertyTourEligible,vrModel.cdnHost,vrModel.revisionId,vrModel.vrModelGuid,zestimateDeepDiveData.zmaModel,zillowOfferMarket.code,zipcodeSearchUrl.path
0,4713 3rd Pl NW,None,8,"[{'subjectType': None, 'url': 'https://photos....",None,None,None,For Sale by Agent,Greysteel Company LLC,None,...,"[{'link': {'classString': 'noroute', 'href': '...","[{'classString': 'znav-manage-rentals', 'gaExp...","[{'link': {'href': '/user/acct/login/?cid=pf',...",True,None,None,None,None,None,/washington-dc-20011/


In [16]:
url=df_property_detail['hdpUrl'].iloc[0]
zoning= df_property_detail['resoFacts.zoning'].iloc[0]
price=df_property_detail['price'].iloc[0]
rent=df_property_detail['rentZestimate'].iloc[0]
tax_rate=df_property_detail['propertyTaxRate'].iloc[0]

print(' Url:{}'.format( url)) 
print(' Zoning:{}'.format( zoning))
print(' Price:{}'.format( price))
print(' Rent:{}'.format( rent))
print(' tax_rate:{}'.format( tax_rate))

 Url:/homedetails/4713-3rd-Pl-NW-Washington-DC-20011/486979_zpid/
 Zoning:RA-1
 Price:1000000
 Rent:4214
 tax_rate:0.57


In [18]:
# Declares and asks for user to input loan amount. Then converts to float
dp = input('Enter down payment amount \n')
dp = float(dp)

# Declares and asks for user to input loan amount. Then converts to float
loanAmount = input('Enter loan amount \n')
loanAmount = float(loanAmount)

# Declares and asks user to input number of payments in years. Then converts to float. Years * 12 to get
#  total number of months
years = input('How many years will you have the loan? \n')
months = float(years) * 12

# Declares and asks user to input interest rate. Then converts to float and input interest rate is /100/12
interestRate = input('Enter Interest Rate \n')
interestRate = float(interestRate) / 100 / 12

# Formula to calculate monthly payments
mortgagePayment = loanAmount * (interestRate * (1 + interestRate)
                                ** months) / ((1 + interestRate) ** months - 1)

# Prints monthly payment on next line and reformat the string to a float using 2 decimal places
print("The monthly mortgage payment is\n (%.2f) " % mortgagePayment)

# Converts the down payment amount into percent
percent = dp/price * 100
print("The down payment percent is\n (%.2f) " % percent)



Enter down payment amount 
 1000000
Enter loan amount 
 0
How many years will you have the loan? 
 1
Enter Interest Rate 
 5


The monthly mortgage payment is
 (0.00) 
The down payment percent is
 (100.00) 


In [19]:
mortgagePayment

0.0

In [27]:
def net_operating(rent, tax_rate, price):
    
    #Takes input as monthly mortgage amount and monthly rental amount
    #Uses managment expense, amount for repairs, vacancy ratio
    #Example input: net_operating(1000,1,400,200)
    #879.33
    #1000 - 16.67 (tax) - 100 (managment) - 4 (repairs)
    
    mortgage_amt = mortgagePayment
    prop_managment = rent * 0.10
    prop_tax = (price * (tax_rate/100)/12)
    prop_repairs = (price * 0.002)/12
    insurance = (price * .005)/12
    vacancy = (rent*0.02)
    #These sections are a list of all the expenses used and formulas for each
    global net_income
    net_income = rent - prop_managment - prop_tax - prop_repairs - vacancy - insurance
    global CF
    CF = rent - prop_managment - prop_tax - prop_repairs - vacancy - insurance - mortgage_amt
    #Summing up expense
    global output
    output = [prop_managment, prop_tax, prop_repairs, vacancy, net_income]
    print(f'Net Income is:{net_income:.2f}')
    print(output)
    print(f'Mortgage amount is: {mortgage_amt:.2f}')
    print(f'Property management amount is: {prop_managment:.2f}')
    print(f'Property tax amount is:{prop_tax:.2f}')
    print(f'Property repairs amount is:{prop_repairs:.2f}')
    print(f'Insurance amount is:{insurance:.2f}')
    print(f'Cash Flow amount is:{CF:.2f}')
    print(f'vacancy is: {vacancy:.2f}')
    
    return output
    return net_income

net_operating(rent, tax_rate, price)

Net Income is:2649.99
[421.40000000000003, 474.99999999999994, 166.66666666666666, 84.28, 2649.9866666666667]
Mortgage amount is: 0.00
Property management amount is: 421.40
Property tax amount is:475.00
Property repairs amount is:166.67
Insurance amount is:416.67
Cash Flow amount is:2649.99
vacancy is: 84.28


[421.40000000000003,
 474.99999999999994,
 166.66666666666666,
 84.28,
 2649.9866666666667]

In [28]:
net_operating(rent, tax_rate, price)
output

Net Income is:2649.99
[421.40000000000003, 474.99999999999994, 166.66666666666666, 84.28, 2649.9866666666667]
Mortgage amount is: 0.00
Property management amount is: 421.40
Property tax amount is:475.00
Property repairs amount is:166.67
Insurance amount is:416.67
Cash Flow amount is:2649.99
vacancy is: 84.28


[421.40000000000003,
 474.99999999999994,
 166.66666666666666,
 84.28,
 2649.9866666666667]

In [29]:
def cap_rate(net_income, price):
    #This function takes net income, and price and calculates the cap rate
    #
    global cap_rate
    cap_rate = ((net_income*12) / price)*100
    print(f'Cap rate is: {cap_rate:.3}','%')

cap_rate(net_income, price)

Cap rate is: 3.18 %


In [30]:
def cash_on_cash(CF, dp):
    cash_return = ((CF*12)/dp)*100
    print(f'Cash on cash is: {cash_return:.3}','%')

cash_on_cash(net_income, dp)


Cash on cash is: 3.18 %


In [22]:
def mortgage_monthly(price,years,percent):
    
    
    #This implements an approach to finding a monthly mortgage amount from the purchase price,
    #years and percent. 
    #Sample input: (300000,20,4) = 2422
    #
    
    
    percent = percent /100
    down = down_payment(price,20)
    loan = price - down
    months = years*12
    interest_monthly = percent/12
    interest_plus = interest_monthly + 1
    exponent = (interest_plus)**(-1*months)
    subtract = 1 - exponent
    division = interest_monthly / subtract
    payment = division * loan
    
    print(f'interest monthly is: {interest_monthly:.2f}')
    print(f'interst_plus amount is: {interest_plus:.2f}')
    print(f'payment amount is:{payment:.2f}')
        
    return(payment)



In [24]:
def down_payment(price,percent):
    #This function takes the price and the downpayment rate and returns the downpayment amount 
    #Ie down_payment(100,20) returns 20
    amt_down = price*(percent/100)
    print(amt_down)
    return(amt_down)

In [ ]:
url = "https://zillow56.p.rapidapi.com/search"

querystring = {"location":"houston, tx"}

headers = {
	"X-RapidAPI-Key": "dae4723a83mshf59232f5dbe9677p1e54aejsn5b66616ebe0b",
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)


#print(response.text)
json.dumps(response.text, indent=4, sort_keys=True)

In [ ]:
property_address='2904 2nd St SE, Washington, DC 20032'
query = property_address + ' zillow home details'
print('Search this phrase in Google Search:', query)

In [ ]:
search_results=search(query, tld='com', lang='en', num=3, start=0, stop=3, pause=0)
search_results_list = [u for u in search_results]
search_results_list

In [ ]:
url = search_results_list[0]
url

In [ ]:
zpid=[x for x in url.split('/')if 'zpid' in x][0].split('_')[0]
print ('Zpid of the property is:',{zpid})

In [ ]:
url = "https://zillow56.p.rapidapi.com/property"

querystring = {"zpid":zpid}

headers = {
	"X-RapidAPI-Key": "dae4723a83mshf59232f5dbe9677p1e54aejsn5b66616ebe0b",
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

response.json()

In [ ]:
df_property_detail = pd.json_normalize(data=response.json())
print('Num of rows:', len(df_property_detail))
print ('Num of columns:',len(df_property_detail.columns))
df_property_detail.head()

In [ ]:
url=df_property_detail['hdpUrl'].iloc[0]
zoning= df_property_detail['resoFacts.zoning'].iloc[0]
price=df_property_detail['price'].iloc[0]
rent=df_property_detail['rentZestimate'].iloc[0]
tax_rate=df_property_detail['propertyTaxRate'].iloc[0]

print(' Url:{}'.format( url)) 
print(' Zoning:{}'.format( zoning))
print(' Price:{}'.format( price))
print(' Rent:{}'.format( rent))
print(' tax_rate:{}'.format( tax_rate))

In [ ]:
def mortgage_monthly(price,years,percent):
    
    
    #This implements an approach to finding a monthly mortgage amount from the purchase price,
    #years and percent. 
    #Sample input: (300000,20,4) = 2422
    #
    
    
    percent = percent /100
    down = down_payment(price,20)
    loan = price - down
    months = years*12
    interest_monthly = percent/12
    interest_plus = interest_monthly + 1
    exponent = (interest_plus)**(-1*months)
    subtract = 1 - exponent
    division = interest_monthly / subtract
    payment = division * loan
    print(payment)
    
    return(payment)

mortgage_monthly(350000,30,20)

In [ ]:
def net_operating(rent, tax_rate, price):
    
    #Takes input as monthly mortgage amount and monthly rental amount
    #Uses managment expense, amount for repairs, vacancy ratio
    #Example input: net_operating(1000,1,400,200)
    #879.33
    #1000 - 16.67 (tax) - 100 (managment) - 4 (repairs)
    
    mortgage_amt = mortgage_monthly(price,20,3)
    prop_management = rent * 0.10
    prop_tax = (price * (tax_rate/100)/12)
    prop_repairs = (price * 0.02)/12
    vacancy = (rent*0.02)
    #These sections are a list of all the expenses used and formulas for each
    
    net_income = rent - prop_management - prop_tax - prop_repairs - vacancy - mortgage_amt
    #Summing up expenses
    output = [prop_management, prop_tax, prop_repairs, vacancy, net_income]
    
    return output
net_operating(rent, tax_rate, price)
print(net_income)
print(output)

In [ ]:
 net_income = rent - prop_managment - prop_tax - prop_repairs - vacancy - mortgage_amt

In [ ]:
zestimate=df_property detail

In [ ]:
def get_zestimate(address_and_city):
    zestimate_data-api.GetSearchResults(key,address_and_city, postal_code)
    address_dict = zestimate_data.get_dict()
    amount = address_dct['zestimate']['amount']
    amount = locale.currency(amount, grouping=True)
    return amount

In [ ]:
url = "https://zillow-coml.p.rapidapi.com/property"
querystring = {"zpid":zpid}
headers = {
"X-RapidAPI-Host":"zillow56.p.rapidapi.com",
"X_RapidAPI_Key": "dae4723a83mshf59232f5dbe9677p1e54aejsn5b66616ebe0b"
}

In [ ]:
url = "https://zillow56.p.rapidapi.com/property"

querystring = {"zpid":zpid}

headers = {
	"X-RapidAPI-Key": "dae4723a83mshf59232f5dbe9677p1e54aejsn5b66616ebe0b",
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

response.json()

In [ ]:
data=api.GetSearchResults(key, address, postal_code)

In [ ]:
address=""

In [ ]:
def pandas_neighborhood():
    df=pd.read_excel('address.xlsx')
    for index. row in df.iterrows():
        print(row)

In [ ]:
def neighborhood():
    location- "address.xlsx"
    wb = xlrd.open_workbook(location)
    sheet = wb.sheet_by_index(0)
    sheet.cell_value(0,0)
    # Shows how many houses there are less than
    
    for i in range (sheet.nrows):
        address = sheet.cell_value(i,0)
        adress_and_city = "Street" + "Address" + "City"
        
        try:
            zestimate_data - api.GetSerarchResults(X-RapidAPI-Key, address_and_city, postal_code)
            address_dict - zestimate_data.get_dict()
            amount= addres_dict('zestimate')('amount')
            amount - locale.currency(amount, grouping=True)
        except zillow.error.zillowerror:
            print("Error occurred with this address: ")

In [ ]:
def get_zestimate(address_and_city):
    zestimate_data-api.GetSearchResults(key,address_and_city, postal_code)
    address_dict = zestimate_data.get_dict()
    amount - addres_dct['zestimate']['amount']
    amount = locale.currency(amount, grouping=True)
    return amount

In [ ]:
get_zestimate("127 N. Front St, Mildford")

In [ ]:
pp.pprint(deep_results,get_dict[])

In [ ]:
my_dict = deep_results.get_dict[]

In [ ]:
Amount = my_dict['zestimate']['amount']

In [ ]:
print("Your house is worth" = locale.currency(amount, "en_US"=True))

In [ ]:
def

In [ ]:
pd.options.display.max_columns = None

In [ ]:
url = "https://zillow56.p.rapidapi.com/property"

def price(url):
    querystring = {"zpid":"7594920"}

    headers = {
        "X-RapidAPI-Key": "dae4723a83mshf59232f5dbe9677p1e54aejsn5b66616ebe0b",
        "X-RapidAPI-Host": "zillow56.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    print(response.text(price))
    

price(url)

In [ ]:
import requests

url = "https://zillow56.p.rapidapi.com/search_url"

querystring = {"url":"https://www.zillow.com/homes/for_sale/2_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A2%7D%2C%22mapBounds%22%3A%7B%22west%22%3A-112.39143704189931%2C%22east%22%3A-110.78468655361806%2C%22south%22%3A32.79032628812945%2C%22north%22%3A33.7227901388417%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D","page":"3"}

headers = {
	"X-RapidAPI-Key": "dae4723a83mshf59232f5dbe9677p1e54aejsn5b66616ebe0b",
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup{response.text, 'xml'}

In [ ]:
print (soup.prettify())

In [ ]:
print(soup.find('').prettify())

In [ ]:
soup.find('').find('').text

In [ ]:
def zillow_parse_search_results(response_content):
    soup = ButifulSoup(get_url, 'xml')
    
    content_dict = {
    
    
    }
    return pd.DataFrame.from_dict(content_dict)

In [ ]:
df = zillow_parse_search_results(response_content=get_searchresults_response)
df